In [2]:
import Pkg
Pkg.add("ReinforcementLearning")
Pkg.add("Lux")
Pkg.add("Random")
Pkg.add("Flatten")
Pkg.add("Plots")
Pkg.add("NNlib")

    Updating registry at `C:\Users\datapaf\.julia\registries\General.toml`


   Resolving package versions...

  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Manifest.toml`


...

  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...

  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\datapaf\.julia\environments\v1.8\Manifest.toml`


In [3]:
using ReinforcementLearning #Environments and API
using Lux #Neural networks with explicit parameterization (Flux would also work) To install: ] add https://github.com/avik-pal/Lux.jl
using Random #RNG generation
using Flatten #Flatten structs of parameters in vector of arrays - simplifies equations
using Plots #Basic plots
using NNlib #More neural network functionality

In [4]:
struct PGAgent{A, B, C, D, E, F}<:AbstractPolicy #A B C D E F sets static types so they are known at compile time - performance boost
    f::A #::A same as above
    θ::B
    st::C
    params::D
    trace::E
    baseline::F #Baseline
end

In [5]:
mutable struct EWMA{A, B} #Allows averaging without storing all rewards. See https://en.wikipedia.org/wiki/Moving_average#Exponential_moving_average
    mean::A
    α::B #Controlls decay rate of average
end
function update!(s::EWMA, x)
    s.mean = (1.0 - s.α) *  s.mean + s.α * x
end
function (s::EWMA)()
    return s.mean
end

In [6]:
mutable struct PGParams{A, B, C} #Learning parameters
    γ::A #Discount rate
    α::B #Step size
    σ::C #Std
end

In [7]:
function (me::PGAgent)(env)
    #Action and learning loop (merged)
    #Get list of parameters
    flatθ = Flatten.flatten(me.θ, Array)
    #Get reward
    r = reward(env)

    #Update parameters with reward and trace
    for i = 1:length(flatθ)
        flatθ[i] .= flatθ[i] .+ me.params.α * (r - me.baseline()) .*  me.trace[i] ./ me.params.σ^2 #The flattened vector is a list of pointers to the arrays in θ. Changing the contect of flatθ with .= also changes θ.
    end
    #Update baseline
    update!(me.baseline, r)
    #Generate parameters for this iteration
    noisyθ = deepcopy(me.θ)
    flatnosiyθ = Flatten.flatten(noisyθ, Array)
    #Generate noise
    n = [me.params.σ * randn(size(x)) for x in flatnosiyθ]
    #Add noise to parameters parameters
    for i in 1:length(flatnosiyθ)
        flatnosiyθ[i] .=  flatnosiyθ[i] .+ n[i]
    end
    #Update trace with noise
    for i in 1:length(me.trace)
        me.trace[i] .= me.params.γ * me.trace[i] + (1-me.params.γ) * n[i]
    end
    return 2.0*me.f(state(env), noisyθ, me.st)[1][1] #Return action from noisy policy
end

In [8]:
#Init environment
env = PendulumEnv(continuous = true)
action_space(env)

-2.0..2.0

In [28]:
#Create ANN and save parameter struct and state (of the ANN in case of RNN, LSTM etc - otherwise empty struct) struct
rng = Random.default_rng()
nhidden = 4
#model = Chain(Dense(length(state(env)) => nhidden,tanh), Dense(nhidden => 1,tanh)) #Creates an ANN
model = Chain(Dense(length(state(env)) => nhidden,sigmoid), Dense(nhidden => 1,sigmoid)) #Creates an ANN
ps, st = Lux.setup(rng, model)

((layer_1 = (weight = Float32[0.55023634 0.162633 -0.51396894; -0.6010354 -0.5424204 0.6072771; 0.35540164 0.5403466 0.91840017; -0.5179416 -0.893431 -0.08558017], bias = Float32[0.0; 0.0; 0.0; 0.0;;]), layer_2 = (weight = Float32[-0.8809461 -0.006522571 0.05020387 -0.90532714], bias = Float32[0.0;;])), (layer_1 = NamedTuple(), layer_2 = NamedTuple()))

In [29]:
#Init trace vector
tr = deepcopy(ps)
fltr = Flatten.flatten(tr, Array) #Flatten so that we can iterate easily over all parameters in a vector of arrays instead of through the struct
[x .= 0.0 .* x for x in fltr] #Set initial traces to 0

4-element Vector{Matrix{Float32}}:
 [0.0 0.0 -0.0; -0.0 -0.0 0.0; 0.0 0.0 0.0; -0.0 -0.0 -0.0]
 [0.0; 0.0; 0.0; 0.0;;]
 [-0.0 -0.0 0.0 -0.0]
 [0.0;;]

In [30]:
#params = PGParams(0.95, 0.00001, 0.1) #Set learning parameters
params = PGParams(0.95, 0.01, 0.1) #Set learning parameters
agent = PGAgent(model,ps,st,params,fltr, EWMA(-0.0, 0.001)) #Create agent

typename(PGAgent)
├─ f => typename(Chain)
│  └─ layers => typename(NamedTuple)
│     ├─ layer_1 => typename(Dense)
│     │  ├─ activation => typename(typeof(sigmoid_fast))
│     │  ├─ in_dims => 3
│     │  ├─ out_dims => 4
│     │  ├─ init_weight => typename(typeof(Lux.glorot_uniform))
│     │  └─ init_bias => typename(typeof(Lux.zeros32))
│     └─ layer_2 => typename(Dense)
│        ├─ activation => typename(typeof(sigmoid_fast))
│        ├─ in_dims => 4
│        ├─ out_dims => 1
│        ├─ init_weight => typename(typeof(Lux.glorot_uniform))
│        └─ init_bias => typename(typeof(Lux.zeros32))
├─ θ => typename(NamedTuple)
│  ├─ layer_1 => typename(NamedTuple)
│  │  ├─ weight => 4×3 Matrix{Float32}
│  │  └─ bias => 4×1 Matrix{Float32}
│  └─ layer_2 => typename(NamedTuple)
│     ├─ weight => 1×4 Matrix{Float32}
│     └─ bias => 1×1 Matrix{Float32}
├─ st => typename(NamedTuple)
│  ├─ layer_1 => typename(NamedTuple)
│  └─ layer_2 => typename(NamedTuple)
├─ params => typename(PGParams)


In [31]:
#Test model and full agent
model([1; 0;1], ps, st)
agent(env)

0.880697756705224

In [32]:
agent.θ
#Training loop
run(
    agent,
    env,
    #StopAfterEpisode(100000),
    StopAfterEpisode(1000),
    TotalRewardPerEpisode()
)

               ⠀⠀⠀⠀⠀⠀⠀⠀⠀Total reward per episode⠀⠀⠀⠀⠀⠀⠀⠀⠀ 
               ┌────────────────────────────────────────┐ 
             0 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
               │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
               │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
               │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀⠀⡀⠀⢰⠀⠀⠀⠀⡇⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
               │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⣄⠀⣄⡇⠀⢸⠀⠀⠀⠀⡇⢀⣼⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
               │⠀⠀⠀⡄⠀⠀⠀⠀⠀⠀⡇⣿⡇⣿⣿⣤⢸⠀⠀⠀⠀⣧⢸⣿⡇⢠⣤⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣄│ 
               │⠀⠀⢰⡇⠀⠀⠀⠀⠀⢀⣧⣿⡇⣿⣿⣿⢸⠀⠀⠀⠀⣿⢸⣿⡇⢸⣿⣧⠀⠀⡀⠀⠀⠀⠀⢸⠀⢠⢠⣿│ 
   Score       │⡇⣆⣼⡇⠀⠀⠀⠀⠀⢸⣿⣿⡇⣿⣿⣿⢸⡇⠀⠀⠀⣿⢸⣿⡇⢸⣿⣿⣆⢀⡇⠀⡄⠀⢀⢸⡆⢸⣾⣿│ 
               │⣧⣿⣿⡇⠀⠀⠀⠀⠀⢸⣿⣿⡇⣿⣿⣿⢸⡇⠀⠀⢠⣿⢸⣿⡇⢸⣿⣿⣿⢸⣇⣤⡇⠀⢸⣿⣷⣾⣿⣿│ 
               │⣿⣿⣿⡇⡄⠀⡄⡄⢀⢸⣿⣿⡇⣿⣿⣿⢸⡇⠀⠀⢸⣿⢸⣿⡇⢸⣿⣿⣿⣿⣿⣿⣇⣶⣿⣿⣿⣿⣿⣿│ 
               │⣿⣿⣿⣿⣿⣇⣧⣿⣾⣿⣿⣿⣧⣿⣿⣿⣸⣷⢠⡄⣾⣿⢸⣿⡇⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ 
               │⣿⡟⠛⠛⠛⣿⠛⠛⠛⣿⣿⡟⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⣸⣿⡇⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿│ 
               │⠛⠃⠀⠀⠀⣿⠀⠀⠀⠹⡿⡇⣿⣿⡇⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢻⢹⢻⣿⣿⢿⣿⣿⣿⣿⣿│ 
               │⠀⠀⠀⠀⠀⣿⠀⠀⠀⠀⠁⠀⣿⣿⡇⠿⢿⢻⣿⣿⣿⣿⣿⡿⠿⡟⣿⣿⢸⠘⠸⠀⠀⠀⠈⠛⣿⣿⡟⡿│ 
         -2000 │⠀⠀⠀⠀⠀⠹⠀⠀⠀⠀⠀⠀⠏⠟⠀⠀⠈⠈⠉⠋⠹⠏⠛⠁⠀⠁⠋⠈⠀⠀⠀⠀⠀⠀⠀⠀⠃⠀⠀⠁

TotalRewardPerEpisode([-1283.809995083607, -1471.0376161826691, -1454.278483612224, -1593.9567473557097, -1007.8448823254489, -1336.166957378007, -1345.3689579120419, -1095.4766721401438, -1654.4161037125316, -1178.6025768226602  …  -853.0803667796748, -1223.27619408318, -1191.1668151641375, -1089.562997601789, -774.7814209160039, -840.8911028447534, -1195.4975820400846, -873.8606582720981, -1172.1426094401097, -1527.2263835614935], 0.0, true)

In [14]:
agent.θ
#This should plot something visually - did not work for me
demo = Experiment(agent,
    env,
    StopWhenDone(),
    RolloutHook(plot, closeall),
   "PG <-> Demo")
run(demo)

PG <-> Demo


PG <-> Demo


typename(Experiment)
├─ policy => typename(PGAgent)
│  ├─ f => typename(Chain)
│  │  └─ layers => typename(NamedTuple)
│  │     ├─ layer_1 => typename(Dense)
│  │     │  ├─ activation => typename(typeof(tanh_fast))
│  │     │  ├─ in_dims => 3
│  │     │  ├─ out_dims => 4
│  │     │  ├─ init_weight => typename(typeof(Lux.glorot_uniform))
│  │     │  └─ init_bias => typename(typeof(Lux.zeros32))
│  │     └─ layer_2 => typename(Dense)
│  │        ├─ activation => typename(typeof(tanh_fast))
│  │        ├─ in_dims => 4
│  │        ├─ out_dims => 1
│  │        ├─ init_weight => typename(typeof(Lux.glorot_uniform))
│  │        └─ init_bias => typename(typeof(Lux.zeros32))
│  ├─ θ => typename(NamedTuple)
│  │  ├─ layer_1 => typename(NamedTuple)
│  │  │  ├─ weight => 4×3 Matrix{Float32}
│  │  │  └─ bias => 4×1 Matrix{Float32}
│  │  └─ layer_2 => typename(NamedTuple)
│  │     ├─ weight => 1×4 Matrix{Float32}
│  │     └─ bias => 1×1 Matrix{Float32}
│  ├─ st => typename(NamedTuple)
│  │  ├─ layer